In [1]:
import spacy
from spacy import displacy
from lambeq.backend.drawing import draw
from lambeq.backend.grammar import Cup, Id, Ty, Word
from lambeq import AtomicType, IQPAnsatz, BinaryCrossEntropyLoss, NumpyModel, QuantumTrainer, SPSAOptimizer, Dataset
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
from matplotlib import rcParams

# 型の定義
n, s = Ty('n'), Ty('s')

# 言語モデルのロード
nlp = spacy.load("ja_ginza")

rcParams['font.family'] = 'Hiragino Sans'  # macOS標準のヒラギノ角ゴ
rcParams['axes.unicode_minus'] = False     # マイナス記号が文字化けしないように

In [2]:
df = pd.read_csv("/Users/horiuchiminori/Desktop/研究/datasets/日本語4クラス/4emos_jp_4000.csv")
ovr_cols = pd.get_dummies(df["label"], prefix="label", dtype=int)
df_ovr = pd.concat([df, ovr_cols], axis=1)

train_df, test_df = train_test_split(df_ovr, test_size=0.3, random_state=42, stratify=df['label'])

# トークン分割関数
def tokenize(text):
    return [t.text for t in nlp(text)]

train_tokens = set()

for sentence in train_df["text"]:
    for token in tokenize(sentence):
        train_tokens.add(token)

def test_sentence_is_valid(text):
    tokens = tokenize(text)
    # １つでも train にないトークンがあれば不可
    return all(token in train_tokens for token in tokens)

test_df_filtered = test_df[test_df["text"].apply(test_sentence_is_valid)].reset_index(drop=True)

In [3]:
#test_df_filtered

In [4]:
# 型の割り当て
def assign_types(sentence):
    doc = nlp(sentence)
    pregroup_types = {}
    dependencies = []

    # 文末記号を除いてトークン化
    valid_tokens = [t for t in doc if t.text not in ['.', '。', '!', '！', '?', '？']]

    for token in valid_tokens:
        token_key = f"{token.text}_{token.i}"  # ← spaCyの固有インデックスを使用
        head_key = f"{token.head.text}_{token.head.i}" if token.head.text in [t.text for t in valid_tokens] else None

        # 型割り当て
        if token.dep_ == 'ROOT':
            pregroup_types[token_key] = s
        else:
            pregroup_types[token_key] = n

        if head_key:
            dependencies.append((token_key, head_key))

    # 依存関係による型調整
    for dep_from, dep_to in dependencies:
        if dep_from == dep_to or dep_to not in pregroup_types or dep_from not in pregroup_types:
            continue

        idx1 = int(dep_from.split('_')[-1])
        idx2 = int(dep_to.split('_')[-1])

        if idx1 < idx2:
            pregroup_types[dep_to] = n.r @ pregroup_types[dep_to]
        else:
            pregroup_types[dep_to] = pregroup_types[dep_to] @ n.l

    return pregroup_types

# Diagram作成
def create_diagram(sentence):
    pregroup_types = assign_types(sentence)

    words = []
    types = Ty()

    # ここでインデックスを除去して、lambeqが単語単位でパラメータを共有できるようにする
    for word_key, typ in pregroup_types.items():
        base_word = word_key.split('_')[0]  # "book_0" → "book"
        words.append(Word(base_word, typ))
        types @= typ

    # すべてのWordをテンソル積で連結
    diagram = Id().tensor(*words)

    # Cups適用
    i = 0
    while i < len(types) - 1:
        if types[i:i + 2] == n @ n.r:
            diagram = diagram >> types[:i] @ Cup(n, n.r) @ types[i + 2:]
            types = types[:i] @ types[i + 2:]
            i = max(0, i - 1)
        elif types[i:i + 2] == n.l @ n:
            diagram = diagram >> types[:i] @ Cup(n.l, n) @ types[i + 2:]
            types = types[:i] @ types[i + 2:]
            i = max(0, i - 1)
        else:
            i += 1

    return diagram

In [5]:
# 各データセットのdiagram化
train_sentences = train_df['text']
test_sentences = test_df['text']
train_diagrams = []
test_diagrams = []

for sentence in train_sentences:
    train_diagrams.append(create_diagram(sentence))

for sentence in test_sentences:
    test_diagrams.append(create_diagram(sentence))

In [6]:
# 量子回路設計
ansatz = IQPAnsatz({AtomicType.NOUN: 1, AtomicType.SENTENCE: 1}, n_layers=1, n_single_qubit_params=3)

train_circuits = [ansatz(diagram) for diagram in train_diagrams]
test_circuits = [ansatz(diagram) for diagram in test_diagrams]

all_circuits = train_circuits + test_circuits # モデル初期化時に全量子回路を提供するための準備
model = NumpyModel.from_diagrams(all_circuits, use_jit=True)

# 損失関数と予測精度の定義
bce = BinaryCrossEntropyLoss(use_jax=True)
acc = lambda y_hat, y: np.mean(np.argmax(y_hat, axis=1) == np.argmax(y, axis=1))

In [7]:
# 学習の設定
BATCH_SIZE = 16
LEARNING_RATE = 3e-2
EPOCHS = 100
SEED = 0

In [8]:
def make_binary_labels(y, positive_class):
    """positive_class のとき [1,0]、それ以外 [0,1] にする"""
    return np.array([[1, 0] if label == positive_class else [0, 1] for label in y])

ovr_models = []         # モデルごとの保存
ovr_probabilities = []  # 各クラスの test の positive確率

for cls in range(4):
    print(f"\n==============================")
    print(f"  One-vs-Rest 学習: positive = {cls}")
    print(f"==============================\n")

    train_labels = make_binary_labels(train_df[f"label_{cls}"], cls)
    test_labels = make_binary_labels(test_df[f"label_{cls}"], cls)

    train_dataset = Dataset(train_circuits, train_labels, batch_size=BATCH_SIZE)
    test_dataset = Dataset(test_circuits, test_labels, shuffle=False)

    model = NumpyModel.from_diagrams(all_circuits, use_jit=False)

    trainer = QuantumTrainer(
        model,
        loss_function=bce,
        epochs=EPOCHS,
        optimizer=SPSAOptimizer,
        optim_hyperparams={'a': 0.1, 'c': 0.06, 'A':0.01*EPOCHS},
        evaluate_functions={'acc': acc},
        evaluate_on_train=True,
        verbose='text',
        seed=SEED
    )

    trainer.fit(train_dataset, test_dataset, log_interval=20)

    # positiveクラスの確率を保存
    y_hat = model.forward(test_circuits)
    prob_pos = y_hat[:, 0]     # [1,0] が positive なので y_hat の1列目
    ovr_probabilities.append(prob_pos)

    #モデル保存
    ovr_models.append(model)

# 予測確率をまとめて [num_samples, 4] に
prob_matrix = np.vstack(ovr_probabilities).T

# 各行で最大の確率のインデックス（クラス番号）を取得
pred_labels = np.argmax(prob_matrix, axis=1)
true_labels = test_df['label'].values

# 正解率
accuracy = np.mean(pred_labels == true_labels)
print("Accuracy:", accuracy)


  One-vs-Rest 学習: positive = 0



Epoch 20:   train/loss: 0.8701   valid/loss: 0.9107   train/time: 29m6s   valid/time: 5m10s   train/acc: 0.5888   valid/acc: 0.5712
Epoch 40:   train/loss: 0.9750   valid/loss: 0.8820   train/time: 29m23s   valid/time: 5m12s   train/acc: 0.6156   valid/acc: 0.5675
Epoch 60:   train/loss: 1.0173   valid/loss: 0.8669   train/time: 29m57s   valid/time: 5m8s   train/acc: 0.6475   valid/acc: 0.5984
Epoch 80:   train/loss: 0.6390   valid/loss: 0.8838   train/time: 30m36s   valid/time: 5m33s   train/acc: 0.6557   valid/acc: 0.5857
Epoch 100:  train/loss: 0.6221   valid/loss: 0.8836   train/time: 32m13s   valid/time: 5m38s   train/acc: 0.6720   valid/acc: 0.5938

Training completed!
train/time: 2h31m15s   train/time_per_epoch: 1m31s   train/time_per_step: 0.56s   valid/time: 26m41s   valid/time_per_eval: 16.01s



  One-vs-Rest 学習: positive = 1



Epoch 20:   train/loss: 0.9441   valid/loss: 1.0009   train/time: 33m53s   valid/time: 6m14s   train/acc: 0.5647   valid/acc: 0.5186
Epoch 40:   train/loss: 0.7293   valid/loss: 0.9976   train/time: 35m44s   valid/time: 6m8s   train/acc: 0.5771   valid/acc: 0.4878
Epoch 60:   train/loss: 0.9969   valid/loss: 0.9800   train/time: 36m50s   valid/time: 6m41s   train/acc: 0.6028   valid/acc: 0.5277
Epoch 80:   train/loss: 0.7323   valid/loss: 0.9566   train/time: 39m33s   valid/time: 5m38s   train/acc: 0.6304   valid/acc: 0.5440
Epoch 100:  train/loss: 0.7323   valid/loss: 0.9514   train/time: 40m19s   valid/time: 8m1s   train/acc: 0.6195   valid/acc: 0.5403

Training completed!
train/time: 3h6m19s   train/time_per_epoch: 1m52s   train/time_per_step: 0.69s   valid/time: 32m42s   valid/time_per_eval: 19.62s



  One-vs-Rest 学習: positive = 2



Epoch 20:   train/loss: 0.7040   valid/loss: 1.0214   train/time: 46m10s   valid/time: 6m37s   train/acc: 0.5511   valid/acc: 0.5150
Epoch 40:   train/loss: 0.7736   valid/loss: 0.9791   train/time: 44m47s   valid/time: 7m52s   train/acc: 0.6133   valid/acc: 0.5286
Epoch 60:   train/loss: 0.7141   valid/loss: 0.9526   train/time: 58m17s   valid/time: 8m37s   train/acc: 0.6195   valid/acc: 0.5530
Epoch 80:   train/loss: 0.7916   valid/loss: 0.9228   train/time: 1h13m56s   valid/time: 4m49s   train/acc: 0.6405   valid/acc: 0.5703
Epoch 100:  train/loss: 0.4741   valid/loss: 0.9259   train/time: 1h18m9s   valid/time: 11m53s   train/acc: 0.6665   valid/acc: 0.5684

Training completed!
train/time: 5h1m20s   train/time_per_epoch: 3m1s   train/time_per_step: 1.12s   valid/time: 39m48s   valid/time_per_eval: 23.88s



  One-vs-Rest 学習: positive = 3



Epoch 20:   train/loss: 0.7040   valid/loss: 1.0214   train/time: 1h17m8s   valid/time: 11m55s   train/acc: 0.5511   valid/acc: 0.5150
Epoch 40:   train/loss: 0.7736   valid/loss: 0.9791   train/time: 1h26m4s   valid/time: 12m49s   train/acc: 0.6133   valid/acc: 0.5286
Epoch 60:   train/loss: 0.7141   valid/loss: 0.9526   train/time: 1h37m10s   valid/time: 20m25s   train/acc: 0.6195   valid/acc: 0.5530
Epoch 80:   train/loss: 0.7916   valid/loss: 0.9228   train/time: 1h38m56s   valid/time: 1h6m23s   train/acc: 0.6405   valid/acc: 0.5703
Epoch 100:  train/loss: 0.4741   valid/loss: 0.9259   train/time: 4h24m18s   valid/time: 6m25s   train/acc: 0.6665   valid/acc: 0.5684

Training completed!
train/time: 10h23m36s   train/time_per_epoch: 6m14s   train/time_per_step: 2.32s   valid/time: 1h57m56s   valid/time_per_eval: 1m11s


Accuracy: 0.27379873073436084


In [9]:
"""# ひとつめ
train_y = train_df['label_0']
test_y = test_df['label_0']

# ラベルのone-hot表現
train_labels = np.array([[1, 0] if i == 1 else [0, 1] for i in train_y])
test_labels = np.array([[1, 0] if i == 1 else [0, 1] for i in test_y])

train_dataset = Dataset(train_circuits, train_labels, batch_size=BATCH_SIZE)
test_dataset = Dataset(test_circuits, test_labels, shuffle=False)

# 学習
trainer.fit(train_dataset, test_dataset, log_interval=100)

# positive判定確率
y_hat = model.forward(test_circuits) # one-hotで確率を表示
prob_class_0 = y_hat[:, 0]  # positiveクラスの確率
prob_class_0
"""

"# ひとつめ\ntrain_y = train_df['label_0']\ntest_y = test_df['label_0']\n\n# ラベルのone-hot表現\ntrain_labels = np.array([[1, 0] if i == 1 else [0, 1] for i in train_y])\ntest_labels = np.array([[1, 0] if i == 1 else [0, 1] for i in test_y])\n\ntrain_dataset = Dataset(train_circuits, train_labels, batch_size=BATCH_SIZE)\ntest_dataset = Dataset(test_circuits, test_labels, shuffle=False)\n\n# 学習\ntrainer.fit(train_dataset, test_dataset, log_interval=100)\n\n# positive判定確率\ny_hat = model.forward(test_circuits) # one-hotで確率を表示\nprob_class_0 = y_hat[:, 0]  # positiveクラスの確率\nprob_class_0\n"

In [10]:
for idx, (sentence, true, pred) in enumerate(zip(test_sentences, true_labels, pred_labels)):
    if pred != true:
        print(f" 文: {sentence}")
        print(f" 予測ラベル: {pred}")
        print(f" 正解ラベル: {true}")

 文: 予想の斜め上の展開に絶句。
 予測ラベル: 0
 正解ラベル: 3
 文: 陰口を聞いてしまって最悪だ。
 予測ラベル: 0
 正解ラベル: 1
 文: はらわたが煮えくり返りそうだ。
 予測ラベル: 0
 正解ラベル: 1
 文: 濡れた靴下で歩くのが不快だ。
 予測ラベル: 0
 正解ラベル: 1
 文: こめかみの血管が脈打つ。
 予測ラベル: 2
 正解ラベル: 1
 文: 好きなバンドが活動休止した。
 予測ラベル: 3
 正解ラベル: 2
 文: 家族から温かい手紙をもらった。
 予測ラベル: 1
 正解ラベル: 0
 文: 子供の成長を感じて胸が熱い。
 予測ラベル: 1
 正解ラベル: 0
 文: 新しい服に袖を通し胸が弾む。
 予測ラベル: 1
 正解ラベル: 0
 文: 水槽の外から世界を見てる。
 予測ラベル: 3
 正解ラベル: 2
 文: 店員の態度が悪く気分を害す。
 予測ラベル: 2
 正解ラベル: 1
 文: 育てた花が枯れてしまい凹む。
 予測ラベル: 1
 正解ラベル: 2
 文: 蚊取り線香が効かず刺された。
 予測ラベル: 2
 正解ラベル: 1
 文: 指先まで温かい血が巡る。
 予測ラベル: 2
 正解ラベル: 0
 文: 鏡の自分が違う動きをした。
 予測ラベル: 0
 正解ラベル: 3
 文: 真夏に雹が降り出し目を疑う。
 予測ラベル: 0
 正解ラベル: 3
 文: 鏡の自分が笑った。
 予測ラベル: 2
 正解ラベル: 3
 文: 料理に髪の毛が入ってて最悪。
 予測ラベル: 3
 正解ラベル: 1
 文: 予想の斜め上すぎて笑う。
 予測ラベル: 0
 正解ラベル: 3
 文: 蝉が部屋に入ってきてパニック。
 予測ラベル: 0
 正解ラベル: 3
 文: ネタバレ踏んでマジで萎えた。
 予測ラベル: 0
 正解ラベル: 1
 文: 見たことない鳥が庭にいて謎。
 予測ラベル: 0
 正解ラベル: 3
 文: 騒音がうるさくて通報した。
 予測ラベル: 0
 正解ラベル: 1
 文: マナーの悪さに辟易している。
 予測ラベル: 2
 正解ラベル: 1
 文: 舌打ちされて喧嘩腰になった。
 予測ラベル: 0
 正解ラベル: 1
 文: 給料が上がって飛び跳ねたい。
 予測ラベル: 2
 正解ラベル: 